# How to run a semantic plugins from file
Now that you're familiar with Kernel basics, let's see how the kernel allows you to run Semantic Plugins and Semantic Functions stored on disk. 

A Semantic Plugin is a collection of Semantic Functions, where each function is defined with natural language that can be provided with a text file. 

Refer to our [glossary](https://github.com/microsoft/semantic-kernel/blob/main/docs/GLOSSARY.md) for an in-depth guide to the terms.

The repository includes some examples under the [samples](https://github.com/microsoft/semantic-kernel/tree/main/samples) folder.

For instance, [this](../../plugins/FunPlugin/Joke/skprompt.txt) is the **Joke function** part of the **FunPlugin plugin**:

```
WRITE EXACTLY ONE JOKE or HUMOROUS STORY ABOUT THE TOPIC BELOW.
JOKE MUST BE:
- G RATED
- WORKPLACE/FAMILY SAFE
NO SEXISM, RACISM OR OTHER BIAS/BIGOTRY.
BE CREATIVE AND FUNNY. I WANT TO LAUGH.
+++++
{{$input}}
+++++
```

Note the special **`{{$input}}`** token, which is a variable that is automatically passed when invoking the function, commonly referred to as a "function parameter". 

We'll explore later how functions can accept multiple variables, as well as invoke other functions.


In the same folder you'll notice a second [config.json](../../plugins/FunPlugin/Joke/config.json) file. The file is optional, and is used to set some parameters for large language models like Temperature, TopP, Stop Sequences, etc.

```
{
  "schema": 1,
  "type": "completion",
  "description": "Generate a funny joke",
  "completion": {
    "max_tokens": 500,
    "temperature": 0.5,
    "top_p": 0.5
  }
}
```

Given a semantic function defined by these files, this is how to load and use a file based semantic function.

Load and configure the kernel, as usual, loading also the AI service settings defined in the [Setup notebook](00-getting-started.ipynb):

In [ ]:
!python -m pip install semantic-kernel==0.5.1.dev0

In [ ]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import (
    AzureChatCompletion,
    OpenAIChatCompletion,
)

kernel = sk.Kernel()

useAzureOpenAI = False

# Configure AI service used by the kernel
if useAzureOpenAI:
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    azure_chat_service = AzureChatCompletion(
        deployment_name="turbo", endpoint=endpoint, api_key=api_key
    )  # set the deployment name to the value of your chat model
    kernel.add_chat_service("chat_completion", azure_chat_service)
else:
    api_key, org_id = sk.openai_settings_from_dot_env()
    oai_chat_service = OpenAIChatCompletion(ai_model_id="gpt-3.5-turbo", api_key=api_key, org_id=org_id)
    kernel.add_chat_service("chat-gpt", oai_chat_service)

Import the plugin and all its functions:

In [ ]:
# note: using plugins from the samples folder
plugins_directory = "../../samples/plugins"

funFunctions = kernel.import_semantic_plugin_from_directory(plugins_directory, "FunPlugin")

jokeFunction = funFunctions["Joke"]

How to use the plugin functions, e.g. generate a joke about "*time travel to dinosaur age*":

In [ ]:
result = await jokeFunction.invoke("travel to dinosaur age")
print(result)

Great, now that you know how to load a plugin from disk, let's show how you can [create and run a semantic function inline.](./03-semantic-function-inline.ipynb)